In [1]:
from plotly import express as px
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm

dct = dict()
main = ''
for file in os.listdir("./assets"):
    if "Gennadi, 2022-11-03" in file:
        print(file)
        main = file
        df_sup = pd.read_csv('./assets/'+file,index_col=(0,1)).replace('-', np.NaN)
#         df_sup['mainDistance'] = [eval(i[0])*1000 if i[1]=='km' else eval(i[0]) for i in df_sup['mainDistance'].str.split(' ')]
        dct[file] = df_sup

print('Files loaded:', len(dct))

df = dct[main]


/Users/lazaros/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/lazaros/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


Gennadi, 2022-11-03.csv
Files loaded: 1


In [2]:

season = []
for i in df.index:
    mo = int(i[0].split('-')[1])
    if mo <=3:
        season.append(1)
    elif mo<=6:
        season.append(2)
    elif mo<=9:
        season.append(3)
    else:
        season.append(4)
df['season'] = season

df['isSustainable'] = df['isSustainable'].astype(int) 
df['propSize'] = df['propSize'].replace({'1,000':'1000'}).astype(float)
df['propSizeChar'] = [('Small' if i<70 else ('Medium' if i<130 else 'Large')) for i in df['propSize']]

df['pricePerDay'] = df['finalPrice'].astype(float)/7#/df['nbMaxAdults']

df = df[df['propDescription'].isin(['Entire villa','Entire holiday home','Entire apartment'])]

df = df[df['reviewsCount']>0]

df['totalScoreChar'] = [('Bad' if i<7.5 else ('Moderate' if i<8.5 else 'Good')) for i in df['totalScore']]

In [3]:
all_prop = df['finalPrice'].unstack().mean(axis=1)
big_prop = df[df['propSize'].astype(float)>130]['finalPrice'].unstack().mean(axis=1)
mid_prop = df[(df['propSize'].astype(float)>=100) & (df['propSize'].astype(float)<=130)]['finalPrice'].unstack().mean(axis=1)
small_prop = df[df['propSize'].astype(float)<100]['finalPrice'].unstack().mean(axis=1)
max_prop = df['finalPrice'].unstack().max(axis=1)
median_prop = df['finalPrice'].unstack().median(axis=1)
min_prop = df['finalPrice'].unstack().min(axis=1)
prices = pd.concat([all_prop,big_prop,mid_prop,small_prop, min_prop,max_prop,median_prop],axis=1)
prices.columns = ['Mean','Big','Mid','Small','Min','Max','Median']
px.line(prices[['Mean','Big','Mid','Small','Max','Min','Median']])

In [57]:
display(
px.box(df[df['season']!=1],x='propSizeChar',y='finalPrice'),
px.box(df[df['season']!=1],x='propDescription',y='finalPrice'),
px.box(df[df['season']!=1],x='isSustainable',y='finalPrice'),
px.box(df[df['season']!=1],x='totalScoreChar',y='finalPrice'),
)

In [31]:
# Number of beds
ls = ['pricePerDay','propSize','geoDistanceMeters','nbAllBeds', 'nbBathrooms', 'nbBedrooms', 
      'nbMaxAdults', 'totalScore', 'reviewsCount']

for i in ls:
        display(i, px.bar(df[i].unstack().astype(float).mean().sort_values()))

'pricePerDay'

'propSize'

'geoDistanceMeters'

'nbAllBeds'

'nbBathrooms'

'nbBedrooms'

'nbMaxAdults'

'totalScore'

'reviewsCount'

In [8]:
target = df[['pricePerDay','geoDistanceMeters','propSize','nbBedrooms',
             'nbAllBeds','isSustainable','totalScore','season','propSizeChar']]

target.index = [(i[0]+':'+i[1]) for i in target.index]
ff = target.dropna()

In [9]:
dim = [i for i in target.columns if i!='propSizeChar']
dim

['pricePerPerson',
 'geoDistanceMeters',
 'propSize',
 'nbBedrooms',
 'nbAllBeds',
 'isSustainable',
 'totalScore',
 'season']

In [10]:
# Create arrays for the features and the response variable
y = ff['pricePerDay'].values
cols = [i for i in ff.columns if i not in['pricePerDay', 'propSizeChar', 'nbBedrooms']]
X = ff[cols].values


In [11]:
mod = sm.OLS(y, X)
res = mod.fit()

In [12]:
print(res.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.829
Model:                            OLS   Adj. R-squared (uncentered):              0.828
Method:                 Least Squares   F-statistic:                              1577.
Date:                Sun, 23 Oct 2022   Prob (F-statistic):                        0.00
Time:                        15:26:32   Log-Likelihood:                         -11938.
No. Observations:                1963   AIC:                                  2.389e+04
Df Residuals:                    1957   BIC:                                  2.392e+04
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [13]:
res.params

array([-0.08837161,  0.6192274 ,  8.46713052,  9.31167304, 11.85192713,
        9.38847628])